## Processing e Merge dei dataset

In [1]:
import datetime as dt
import pandas as pd
import pandas_datareader.data as dd
from pymongo import MongoClient
import pymongo
import pandas as pd
pd.options.mode.chained_assignment = None


client=MongoClient()
db=client.dataman
alpha = db.alpha
#scegliamo di filtrare le serie storiche per il lasso di tempo relativo al primo semestre del 2020
start = dt.datetime(2020,1,2)
end = dt.datetime(2020,6,2)
#importiamo i dati del covid da mongo su python
from pymongo import MongoClient
import pymongo
client=MongoClient()
db=client.dataman
covid = db.covid
c19 = pd.DataFrame(list(covid.find()))
del c19['_id']
#utilizziamo un mask per poter selezionare i dati di interesse relativi al primo semestre
mask = (c19['Date'] > '2020-01-22') & (c19['Date'] <= '2020-06-02')
covid_19 = c19.loc[mask].reset_index()
del covid_19['index']
covid_19

# Francia

#import serie storica indice da yahoo finance
cac40 = dd.DataReader('^FCHI', 'yahoo', start, end)
cac40= cac40.reset_index()
cac40['Date']=cac40['Date'].astype(str)
cac40['Date'] = pd.to_datetime(cac40['Date'], format='%Y-%m-%d')
cac40['Symbol']= '^FCHI'
cac40['Country']= 'France'
#aggiungiamo la colonna Rate per poter calcolare ed inserire le variazioni percentuali degli indici
cac40['Rate']=0
i=0
for i in range (len(cac40.Close)-1):
     cac40.Rate.loc[i+1]=((cac40['Close'].loc[i+1]-cac40['Close'].loc[i])/cac40['Close'].loc[i])*100
#filtriamo il dataframe eliminando le colonne che non interessano
cac40= cac40.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='France')
c_fr = covid_19.loc[mask].reset_index()
del c_fr['index']
#calcoliamo il numero di aumento dei contaggi giornalieri
c_fr['Daily_confirmed']=0
i=0
for i in range (len(c_fr.Confirmed)-1):
    c_fr.Daily_confirmed.loc[i]=((c_fr['Confirmed'].loc[i]-c_fr['Confirmed'].loc[i+1]))

i=1
for i in range (1,len(c_fr.Daily_confirmed)-1):
    if c_fr.Daily_confirmed[i]<0:
        c_fr.Daily_confirmed[i]=(c_fr.Daily_confirmed[i+1]+c_fr.Daily_confirmed[i-1])/2
    else:
            continue
c_fr.Daily_confirmed = c_fr.Daily_confirmed.astype(int)
g= pd.merge(cac40,c_fr,on=['Date','Country'],how='outer')
france=g[g['Close'].notna()].fillna(0)

#Germania
mask = (covid_19['Country']=='Germany')
c_germany = covid_19.loc[mask].reset_index()
del c_germany['index']
dax = dd.DataReader('^GDAXI', 'yahoo', start, end)
dax= dax.reset_index()
dax['Date']=dax['Date'].astype(str)
dax['Date'] = pd.to_datetime(dax['Date'], format='%Y-%m-%d')
dax['Symbol']= '^GDAXI'
dax['Country']= 'Germany'
dax['Rate']=0
i=0
for i in range (len(dax.Close)-1):
     dax.Rate.loc[i+1]=((dax['Close'].loc[i+1]-dax['Close'].loc[i])/dax['Close'].loc[i])*100
        
dax= dax.drop(columns =['High','Low','Open','Volume','Adj Close'])
c_germany['Daily_confirmed']=0
i=0
for i in range (len(c_germany.Confirmed)-1):
    c_germany.Daily_confirmed.loc[i]=((c_germany['Confirmed'].loc[i]-c_germany['Confirmed'].loc[i+1]))
g= pd.merge(dax,c_germany,on=['Date','Country'],how='outer')
germany=g[g['Close'].notna()].fillna(0)

#Giappone
nikkei = dd.DataReader('^N225', 'yahoo', start, end)
nikkei= nikkei.reset_index()
nikkei['Date']=nikkei['Date'].astype(str)
nikkei['Date'] = pd.to_datetime(nikkei['Date'], format='%Y-%m-%d')
nikkei['Symbol']= '^N225'
nikkei['Country']= 'Japan'
nikkei['Rate']=0
i=0
for i in range (len(nikkei.Close)-1):
     nikkei.Rate.loc[i+1]=((nikkei['Close'].loc[i+1]-nikkei['Close'].loc[i])/nikkei['Close'].loc[i])*100
nikkei= nikkei.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Japan')
covid_jp = covid_19.loc[mask].reset_index()
del covid_jp['index']
c_jp=covid_jp.sort_values(by=['Date'],ascending=True)
c_jp['Daily_confirmed']=0
i=0
for i in range (len(c_jp.Confirmed)-1):
    c_jp.Daily_confirmed.loc[i]=((c_jp['Confirmed'].loc[i]-c_jp['Confirmed'].loc[i+1]))
    
g= pd.merge(nikkei,c_jp,on=['Date','Country'],how='outer')
japan=g[g['Close'].notna()].fillna(0)


#Spagna
ibex = dd.DataReader('^IBEX', 'yahoo', start, end)
ibex= ibex.reset_index()
ibex['Date']=ibex['Date'].astype(str)
ibex['Date'] = pd.to_datetime(ibex['Date'], format='%Y-%m-%d')
ibex['Symbol']= '^IBEX'
ibex['Country']= 'Spain'
ibex['Rate']=0
i=0
for i in range (len(ibex.Close)-1):
     ibex.Rate.loc[i+1]=((ibex['Close'].loc[i+1]-ibex['Close'].loc[i])/ibex['Close'].loc[i])*100
ibex= ibex.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Spain')
c_sp = covid_19.loc[mask].reset_index()
del c_sp['index']
c_sp['Daily_confirmed']=0
i=0
for i in range (len(c_sp.Confirmed)-1):
    c_sp.Daily_confirmed.loc[i]=((c_sp['Confirmed'].loc[i]-c_sp['Confirmed'].loc[i+1]))

i=1
for i in range (len(c_sp.Daily_confirmed)-1):
    if c_sp.Daily_confirmed[i]<0:
        c_sp.Daily_confirmed[i]=(c_sp.Daily_confirmed[i+1]+c_sp.Daily_confirmed[i-1])/2
    else:
            continue
            
g= pd.merge(ibex,c_sp,on=['Date','Country'],how='outer')
spain=g[g['Close'].notna()].fillna(0)

#Australia
axjo = dd.DataReader('^AXJO', 'yahoo', start, end)
axjo= axjo.reset_index()
axjo['Date']=axjo['Date'].astype(str)
axjo['Date'] = pd.to_datetime(axjo['Date'], format='%Y-%m-%d')
axjo['Symbol']= '^AXJO'
axjo['Country']= 'Australia'
axjo['Rate']=0
i=0
for i in range (len(axjo.Close)-1):
     axjo.Rate.loc[i+1]=((axjo['Close'].loc[i+1]-axjo['Close'].loc[i])/axjo['Close'].loc[i])*100
axjo= axjo.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Australia')
covid_as = covid_19.loc[mask].reset_index()
del covid_as['index']
c_as=covid_as.sort_values(by=['Date'],ascending=True)
c_as['Daily_confirmed']=0
i=0
for i in range (len(c_as.Confirmed)-1):
    c_as.Daily_confirmed.loc[i]=((c_as['Confirmed'].loc[i]-c_as['Confirmed'].loc[i+1]))
    
g= pd.merge(axjo,c_as,on=['Date','Country'],how='outer')
australia=g[g['Close'].notna()].fillna(0).loc[:105].reset_index(drop=True)

#Argentina
merv = dd.DataReader('^MERV', 'yahoo', start, end)
merv= merv.reset_index()
merv['Date']=merv['Date'].astype(str)
merv['Date'] = pd.to_datetime(merv['Date'], format='%Y-%m-%d')
merv['Symbol']= '^MERV'
merv['Country']= 'Argentina'
merv['Rate']=0
i=0
for i in range (len(merv.Close)-1):
     merv.Rate.loc[i+1]=((merv['Close'].loc[i+1]-merv['Close'].loc[i])/merv['Close'].loc[i])*100
merv= merv.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Argentina')
c_argentina = covid_19.loc[mask].reset_index()
del c_argentina['index']
c_argentina['Daily_confirmed']=0
i=0
for i in range (len(c_argentina.Confirmed)-1):
    c_argentina.Daily_confirmed.loc[i]=((c_argentina['Confirmed'].loc[i]-c_argentina['Confirmed'].loc[i+1]))
    
g= pd.merge(merv,c_argentina,on=['Date','Country'],how='outer')
argentina=g[g['Close'].notna()].fillna(0)

#Stati Uniti d'America
nya = dd.DataReader('^NYA', 'yahoo', start, end)
nya= nya.reset_index()
nya['Date']=nya['Date'].astype(str)
nya['Date'] = pd.to_datetime(nya['Date'], format='%Y-%m-%d')
nya['Symbol']= '^NYA'
nya['Country']= 'US'
nya['Rate']=0
i=0
for i in range (len(nya.Close)-1):
     nya.Rate.loc[i+1]=((nya['Close'].loc[i+1]-nya['Close'].loc[i])/nya['Close'].loc[i])*100
nya= nya.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Us')
covid_us = covid_19.loc[mask].reset_index()
del covid_us['index']
c_us=covid_us.sort_values(by=['Date'],ascending=True)
c_us['Daily_confirmed']=0
i=0
for i in range (len(c_us.Confirmed)-1):
    c_us.Daily_confirmed.loc[i]=((c_us['Confirmed'].loc[i]-c_us['Confirmed'].loc[i+1]))
    
g= pd.merge(nya,c_us,on=['Date','Country'],how='outer')
us=g[g['Close'].notna()].fillna(0)

#Canada
gsptse = dd.DataReader('^GSPTSE', 'yahoo', start, end)
gsptse= gsptse.reset_index()
gsptse['Date']=gsptse['Date'].astype(str)
gsptse['Date'] = pd.to_datetime(gsptse['Date'], format='%Y-%m-%d')
gsptse['Symbol']= '^GSPTSE'
gsptse['Country']= 'Canada'
gsptse['Rate']=0
i=0
for i in range (len(gsptse.Close)-1):
     gsptse.Rate.loc[i+1]=((gsptse['Close'].loc[i+1]-gsptse['Close'].loc[i])/gsptse['Close'].loc[i])*100
gsptse= gsptse.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Canada')
covid_ca = covid_19.loc[mask].reset_index()
del covid_ca['index']
c_ca=covid_ca.sort_values(by=['Date'],ascending=True)
c_ca['Daily_confirmed']=0
i=0
for i in range (len(c_ca.Confirmed)-1):
    c_ca.Daily_confirmed.loc[i]=((c_ca['Confirmed'].loc[i]-c_ca['Confirmed'].loc[i+1]))
    
g= pd.merge(gsptse,c_ca,on=['Date','Country'],how='outer')
canada=g[g['Close'].notna()].fillna(0)

#India
bsesn = dd.DataReader('^BSESN', 'yahoo', start, end)
bsesn= bsesn.reset_index()
bsesn['Date']=bsesn['Date'].astype(str)
bsesn['Date'] = pd.to_datetime(bsesn['Date'], format='%Y-%m-%d')
bsesn['Symbol']= '^BSESN'
bsesn['Country']= 'India'
bsesn['Rate']=0
i=0
for i in range (len(bsesn.Close)-1):
     bsesn.Rate.loc[i+1]=((bsesn['Close'].loc[i+1]-bsesn['Close'].loc[i])/bsesn['Close'].loc[i])*100
bsesn= bsesn.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='India')
covid_in = covid_19.loc[mask].reset_index()
del covid_in['index']
c_in=covid_in.sort_values(by=['Date'],ascending=True)
c_in['Daily_confirmed']=0
i=0
for i in range (len(c_in.Confirmed)-1):
    c_in.Daily_confirmed.loc[i]=((c_in['Confirmed'].loc[i]-c_in['Confirmed'].loc[i+1]))
    
g= pd.merge(bsesn,c_in,on=['Date','Country'],how='outer')
india=g[g['Close'].notna()].fillna(0)
#Cina
sse = dd.DataReader('000001.SS','yahoo',start,end)
sse= sse.reset_index()
sse['Date']=sse['Date'].astype(str)
sse['Date'] = pd.to_datetime(sse['Date'], format='%Y-%m-%d')
sse['Symbol']= '000001.SS'
sse['Country']= 'China'
sse['Rate']=0
i=0
for i in range (len(sse.Close)-1):
     sse.Rate.loc[i+1]=((sse['Close'].loc[i+1]-sse['Close'].loc[i])/sse['Close'].loc[i])*100
sse= sse.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='China')
covid_ch = covid_19.loc[mask].reset_index()
del covid_ch['index']
c_ch=covid_ch.sort_values(by=['Date'],ascending=True)
c_ch['Daily_confirmed']=0
i=0
for i in range (len(c_ch.Confirmed)-1):
    c_ch.Daily_confirmed.loc[i]=((c_ch['Confirmed'].loc[i]-c_ch['Confirmed'].loc[i+1]))
    
g= pd.merge(sse,c_ch,on=['Date','Country'],how='outer')
china=g[g['Close'].notna()].fillna(0).loc[:98].reset_index(drop=True)
#Brasile
bvsp = dd.DataReader('^BVSP','yahoo',start,end)
bvsp= bvsp.reset_index()
bvsp['Date']=bvsp['Date'].astype(str)
bvsp['Date'] = pd.to_datetime(bvsp['Date'], format='%Y-%m-%d')
bvsp['Symbol']= '^BVSP'
bvsp['Country']= 'Brazil'
bvsp['Rate']=0
i=0
for i in range (len(bvsp.Close)-1):
     bvsp.Rate.loc[i+1]=((bvsp['Close'].loc[i+1]-bvsp['Close'].loc[i])/bvsp['Close'].loc[i])*100
bvsp= bvsp.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Brazil')
covid_bz = covid_19.loc[mask].reset_index()
del covid_bz['index']
c_bz=covid_bz.sort_values(by=['Date'],ascending=True)
c_bz['Daily_confirmed']=0
i=0
for i in range (len(c_bz.Confirmed)-1):
    c_bz.Daily_confirmed.loc[i]=((c_bz['Confirmed'].loc[i]-c_bz['Confirmed'].loc[i+1]))
    
g= pd.merge(bvsp,c_bz,on=['Date','Country'],how='outer')
brazil=g[g['Close'].notna()].fillna(0)
#Corea del Sud
kospi = dd.DataReader('^KS11','yahoo',start,end)
kospi= kospi.reset_index()
kospi['Date']=kospi['Date'].astype(str)
kospi['Date'] = pd.to_datetime(kospi['Date'], format='%Y-%m-%d')
kospi['Symbol']= '^KS11'
kospi['Country']= 'Korea, South'
kospi['Rate']=0
kospi= kospi.drop([103])
i=0
for i in range (len(kospi.Close)-1):
     kospi.Rate.loc[i+1]=((kospi['Close'].loc[i+1]-kospi['Close'].loc[i])/kospi['Close'].loc[i])*100
kospi= kospi.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Korea, South')
covid_ks = covid_19.loc[mask].reset_index()
del covid_ks['index']
c_ks=covid_ks.sort_values(by=['Date'],ascending=True)
c_ks['Daily_confirmed']=0
i=0
for i in range (len(c_ks.Confirmed)-1):
    c_ks.Daily_confirmed.loc[i]=((c_ks['Confirmed'].loc[i]-c_ks['Confirmed'].loc[i+1]))
    
g= pd.merge(kospi,c_ks,on=['Date','Country'],how='outer')
sk=g[g['Close'].notna()].fillna(0)
#Russia
moex = dd.DataReader('IMOEX.ME','yahoo',start,end)
moex= moex.reset_index()
moex['Date']=moex['Date'].astype(str)
moex['Date'] = pd.to_datetime(moex['Date'], format='%Y-%m-%d')
moex['Symbol']= 'IMOEX.ME'
moex['Country']= 'Russia'
moex['Rate']=0
i=0
for i in range (len(moex.Close)-1):
     moex.Rate.loc[i+1]=((moex['Close'].loc[i+1]-moex['Close'].loc[i])/moex['Close'].loc[i])*100
moex= moex.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='Russia')
covid_rs = covid_19.loc[mask].reset_index()
del covid_rs['index']
c_rs=covid_rs.sort_values(by=['Date'],ascending=True)
c_rs['Daily_confirmed']=0
i=0
for i in range (len(c_rs.Confirmed)-1):
    c_rs.Daily_confirmed.loc[i]=((c_rs['Confirmed'].loc[i]-c_rs['Confirmed'].loc[i+1]))
    
g= pd.merge(moex,c_rs,on=['Date','Country'],how='outer')
russia=g[g['Close'].notna()].fillna(0)

#dataframe presi da investing.com, poichè non reperibili su yahoo finance
#dataframe che abbiamo controllato e che rispettano l'andamento dell'indici con piccole variazioni


#Italia
mib =pd.read_csv(r'C:\Users\ddoci\Downloads\MIB.csv')
mib['Data']=mib['Data'].astype(str)
mib['Data'] = pd.to_datetime(mib['Data'], format='%d.%m.%Y')
mib['Data'] =  pd.to_datetime(mib['Data'], format='%Y.%m.%d')
#rinominiamo le colonne cosicchè rispettino lo stesso schema dei dataframe di yahoo finance
mib.rename(columns={'Ultimo': 'Close', 'Apertura': 'Open','Massimo':'High','Minimo':'Low','Data':'Date'}, inplace=True)
mib['Symbol']= 'FTSEMIB.MI'
#modifichiamo i valori per far si che siano uguali a quelli di yahoo finance, eliminando il"." utilizzato come separatore
#delle migliaia e sostituendo la "," con il "." per il separatore dei decimali
mib['Close']=mib['Close'].str.replace('.', '').str.replace(',', '.').astype(float)
mib['Open']=mib['Open'].str.replace('.', '').str.replace(',', '.').astype(float)
mib['High']=mib['High'].str.replace('.', '').str.replace(',', '.').astype(float)
mib['Low']=mib['Low'].str.replace('.', '').str.replace(',', '.').astype(float)
mib['Rate']=mib['Var. %'].str.replace(',', '.').str.replace('%', '')
mib['Country']='Italy'
mi= mib.drop(columns =['Vol.','Open','High','Low','Var. %'])
mi=mi.sort_values(by='Date').reset_index(drop=True)
mask = (covid_19['Country']=='Italy')
c_it = covid_19.loc[mask].reset_index()
del c_it['index']
c_it['Daily_confirmed']=0
i=0
for i in range (len(c_it.Confirmed)-1):
    c_it.Daily_confirmed.loc[i]=((c_it['Confirmed'].loc[i]-c_it['Confirmed'].loc[i+1]))
    
g= pd.merge(mi,c_it,on=['Date','Country'],how='outer')
italy=g[g['Close'].notna()].fillna(0)

#Arabia Saudita
tasi_ =pd.read_csv(r'C:\Users\ddoci\Downloads\TASI-SR.csv')
tasi_['Data']=tasi_['Data'].astype(str)
tasi_['Data'] = pd.to_datetime(tasi_['Data'], format='%d.%m.%Y')
tasi_['Data'] =  pd.to_datetime(tasi_['Data'], format='%Y.%m.%d')
tasi_.rename(columns={'Ultimo': 'Close', 'Apertura': 'Open','Massimo':'High','Minimo':'Low','Data':'Date'}, inplace=True)
tasi_['Symbol']= '^TASI-SR'
tasi_['Close']=tasi_['Close'].str.replace('.', '').str.replace(',', '.').astype(float)
tasi_['Rate']=tasi_['Var. %'].str.replace(',', '.').str.replace('%', '')
tasi_['Country']='Saudi Arabia'
tasi= tasi_.drop(columns =['Vol.','Open','High','Low','Var. %'])
tasi=tasi.sort_values(by='Date').reset_index(drop=True)
mask = (covid_19['Country']=='Saudi Arabia')
covid_sa = covid_19.loc[mask].reset_index()
del covid_sa['index']
c_sa=covid_sa.sort_values(by=['Date'],ascending=True)
c_sa['Daily_confirmed']=0
i=0
for i in range (len(c_sa.Confirmed)-1):
    c_sa.Daily_confirmed.loc[i]=((c_sa['Confirmed'].loc[i]-c_sa['Confirmed'].loc[i+1]))
    
g= pd.merge(tasi,c_sa,on=['Date','Country'],how='outer')
sa=g[g['Close'].notna()].fillna(0)

#Turchia
bist_ =pd.read_csv(r'C:\Users\ddoci\Downloads\BIST.csv')
bist_['Data']=bist_['Data'].astype(str)
bist_['Data'] = pd.to_datetime(bist_['Data'], format='%d.%m.%Y')
bist_['Data'] = pd.to_datetime(bist_['Data'], format='%Y.%m.%d')
bist= bist_.drop(columns =['Vol.'])
bist_.rename(columns={'Ultimo': 'Close', 'Apertura': 'Open','Massimo':'High','Minimo':'Low','Data':'Date'}, inplace=True)
bist_['Symbol']= '^BIST'
bist_['Close']=bist_['Close'].str.replace('.', '').str.replace(',', '.').astype(float)
bist_['Rate']=bist_['Var. %'].str.replace(',', '.').str.replace('%', '')
bist_['Country']='Turkey'
bist= bist_.drop(columns =['Vol.','Open','High','Low','Var. %'])
bist=bist.sort_values(by='Date').reset_index(drop=True)
bist=bist.drop(bist.index[[104,105,106]])
mask = (covid_19['Country']=='Turkey')
c_ty = covid_19.loc[mask].reset_index()
del c_ty['index']
c_ty['Daily_confirmed']=0
i=0
for i in range (len(c_ty.Confirmed)-1):
    c_ty.Daily_confirmed.loc[i]=((c_ty['Confirmed'].loc[i]-c_ty['Confirmed'].loc[i+1]))
    
g= pd.merge(bist,c_ty,on=['Date','Country'],how='outer')
turkey=g[g['Close'].notna()].fillna(0)


#dataframe disponibile da yahoo finance solo se scaricato dal sito

#Gran Bretagna
ftse_=pd.read_csv(r'C:\Users\ddoci\Downloads\FTSE.csv')
#filtriamo la tabella selezionando i dati relativi al primo semestre del 2020
ftse=ftse_.loc[192:296].reset_index(drop=True)
ftse['Date']=ftse['Date'].astype(str)
ftse['Date'] = pd.to_datetime(ftse['Date'], format='%Y-%m-%d')
ftse['Symbol']= '^FTSE'
ftse['Country']= 'United Kingdom'
ftse['Rate']=0
i=0
for i in range (len(ftse.Close)-1):
     ftse.Rate.loc[i+1]=((ftse['Close'].loc[i+1]-ftse['Close'].loc[i])/ftse['Close'].loc[i])*100
ftse= ftse.drop(columns =['High','Low','Open','Volume','Adj Close'])
mask = (covid_19['Country']=='United Kingdom')
c_uk= covid_19.loc[mask].reset_index()
del c_uk['index']
c_uk['Daily_confirmed']=0
i=0
for i in range (len(c_uk.Confirmed)-1):
    c_uk.Daily_confirmed.loc[i]=((c_uk['Confirmed'].loc[i]-c_uk['Confirmed'].loc[i+1]))
c_uk
    
g= pd.merge(ftse,c_uk,on=['Date','Country'],how='outer')
uk=g[g['Close'].notna()].fillna(0)

#Merge dei dataframe
merged_df = pd.concat([germany,canada,spain,india,france,australia,argentina,japan,us,china,russia,brazil,sk,italy,sa,turkey,uk], sort=True)
index=merged_df.sort_values(by=['Date'],ascending=False).reset_index()
index['Rate'] = index['Rate'].astype(float)
del index['index']
merged = index[['Country','Symbol','Date','Close','Rate','Confirmed','Daily_confirmed']]
final = merged[merged['Close'].notna()].fillna(0).loc[1:1760].reset_index(drop=True)
final

,Country,Symbol,Date,Close,Rate,Confirmed,Daily_confirmed
0,China,000001.SS,2020-06-02,2921.397949,0.204671,84161.0,7.0
1,Japan,^N225,2020-06-02,22325.609375,1.193065,16837.0,50.0
2,Germany,^GDAXI,2020-06-02,12021.280273,3.749342,183879.0,285.0
3,US,^NYA,2020-06-02,12046.410156,1.228294,0.0,0.0
4,Saudi Arabia,^TASI-SR,2020-06-02,7285.230000,-0.050000,89011.0,1869.0
...,...,...,...,...,...,...,...
1755,Argentina,^MERV,2020-01-02,41106.968750,0.000000,0.0,0.0
1756,Spain,^IBEX,2020-01-02,9691.200195,0.000000,0.0,0.0
1757,Canada,^GSPTSE,2020-01-02,17100.000000,0.000000,0.0,0.0
1758,United Kingdom,^FTSE,2020-01-02,7604.299805,0.000000,0.0,0.0
